**About** : This notebook is used to train RNN models.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
cd ../src/

/workspace/kaggle_rsna_lumbar_spine/src


### Imports

In [3]:
import os
import sys
import glob
import json
import torch
import operator
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from tqdm.notebook import tqdm
from sklearn.metrics import *
from collections import Counter
from scipy.stats import spearmanr

warnings.simplefilter(action="ignore", category=UserWarning)
warnings.simplefilter("ignore", FutureWarning)

In [4]:
from util.logger import (
    prepare_log_folder,
    save_config,
    create_logger,
)

from data.dataset import FeatureDataset
from params import *
from data.preparation import *
from util.logger import Config as ConfigInf
from training.main_lvl2 import k_fold
from util.metrics import *

In [5]:
from model_zoo.models_lvl2 import define_model
from training.losses import StudyLoss
from util.metrics import rsna_loss

### Data

In [6]:
df = prepare_data_lvl2()

if "fold" not in df.columns:
    folds = pd.read_csv("../input/train_folded_v1.csv")
    df = df.merge(folds, how="left")

In [7]:
# df = df[df['fold'] == 0].reset_index(drop=True)

In [8]:
EXP_FOLDERS = {
    # "scs_crop": "../logs/2024-08-29/15/",  # 15
    # "nfn_crop": "../logs/2024-08-29/16/",  # 30
    # "scs_crop_coords":  "../logs/2024-08-29/17/",
    # "nfn_crop_coords":  "../logs/2024-08-29/18/",

    # "dh": '../output/oof____cfg_dh_12s4c.pth',  # Darragh preds
    # "dh_2": '../output/oof____cfg_dh_12s4c.pth',  # Darragh preds
    # "ch": '../output/oof_cfg_ch_35.pth',  # Dieter preds
    
    # "crop_2": "../logs/2024-09-11/1/",
    # "dh": "../output/oof____cfg_dh_19a.pth",  # Darragh preds

    "crop": "../logs/2024-09-14/5/",   # coatnet side  <---- best
    "dh": '../output/oof____cfg_dh_12y8.pth',  # Darragh preds
    "dh_2": "../output/oof____cfg_dh_29a2.pth",  # Darragh preds
}

In [9]:
config = ConfigInf(json.load(open(EXP_FOLDERS["crop"] + "config.json", "r")))

In [10]:
df.head(1)

,study_id,series_id,series_description,spinal_canal_stenosis_l1_l2,spinal_canal_stenosis_l2_l3,spinal_canal_stenosis_l3_l4,spinal_canal_stenosis_l4_l5,spinal_canal_stenosis_l5_s1,left_neural_foraminal_narrowing_l1_l2,left_neural_foraminal_narrowing_l2_l3,...,left_subarticular_stenosis_l2_l3,left_subarticular_stenosis_l3_l4,left_subarticular_stenosis_l4_l5,left_subarticular_stenosis_l5_s1,right_subarticular_stenosis_l1_l2,right_subarticular_stenosis_l2_l3,right_subarticular_stenosis_l3_l4,right_subarticular_stenosis_l4_l5,right_subarticular_stenosis_l5_s1,fold
0,4003253,"[702807833, 1054713880, 2448190387]","[Sagittal T2/STIR, Sagittal T1, Axial T2]",0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1


In [11]:
dataset = FeatureDataset(df, EXP_FOLDERS, targets=CLASSES)

In [12]:
for i in tqdm(range(len(dataset))):
    fts, y, _ = dataset[i]
    # for k in fts:
    #     print(k, fts[k].size())
    break

  0%|          | 0/1975 [00:00<?, ?it/s]

In [13]:
for k in fts:
    print(k, fts[k].size())

crop torch.Size([25, 6])
dh torch.Size([25, 3])
dh_2 torch.Size([25, 3])


In [14]:
# plt.figure(figsize=(20, 5))
# plt.subplot(1, 3, 1)
# for i in range(5):
#     plt.plot(fts['ss_aux'].softmax(1)[:, i], label=LEVELS[i])
# plt.legend()

# plt.subplot(1, 3, 2)
# for i in [1, 2]:
#     plt.plot(fts['ss'][:, i], label=f'left_{i}')
# for i in [4, 5]:
#     plt.plot(fts['ss'][:, i], label=f'right_{i - 3}')
# plt.legend()

# # plt.subplot(1, 3, 3)
# # for i in range(5):
# #     plt.plot(fts['ss'][:, 1, i], label=LEVELS[i])
# # plt.legend()

# plt.show()

In [15]:
# for k in fts:
#     print(k, fts[k].shape)

### Model

In [16]:
model = define_model(
    name="simple",
    num_classes=len(CLASSES) * 3,
    ft_dim=[18, 18, 18],
)

In [17]:
x = {k: fts[k].unsqueeze(0) for k in fts}

In [18]:
# pred, _ = model(x)
# pred.size()

In [19]:
# l = StudyLoss()
# l(pred, y.unsqueeze(0))

In [20]:
# rsna_loss(y.unsqueeze(0).numpy(), pred.softmax(2).detach().numpy())

### Training


In [36]:
class Config:
    """
    Parameters used for training
    """
    # General
    seed = 42
    verbose = 1
    device = "cuda"
    save_weights = True
    targets = CLASSES

    # Data
    exp_folders = {
        # "scs_crop_coords": "../logs/2024-09-12/1/",  # 5f -0.005 scs
        # "scs_crop_coords_2": "../logs/2024-10-02/2/",  # 3f -0.005 scs
        
        "scs_crop_coords": "../logs/2024-10-04/34/",  # 5f -0.005 scs
        "scs_crop_coords_2": "../logs/2024-10-04/37/",  # 3f

        "dh": '../output/oof____cfg_dh_15c.pth',  # Darragh preds
        # "dh": '../output/oof____cfg_dh_12y8.pth',  # Darragh preds
        # "dh_2": "../output/oof____cfg_dh_29a2.pth",  # Darragh preds
        "ch": '../output/oof_cfg_ch_35.pth',  # Dieter predsall

        # "crop_2": "../logs/2024-09-13/7/",  # coatnet side
        "crop_2": "../logs/2024-10-04/9/",   # coatnet frameflip  - 0.3842 tta / 0.3843

        # "crop": "../logs/2024-09-14/5/",   # coatnet side  - 0.3848
        "crop": "../logs/2024-10-04/1/",   # coatnet side fix  - 0.3836 <---- best
        # "crop": "../logs/2024-10-04/31/",  # coatnet side fix frameflip - 0.3843
        
        # "crop": "../logs/2024-10-04/10/",   # coatnet vflip  - 0.3846 tta / 0.3849
        # "crop_ax": "../logs/2024-09-30/6/", 
        # "crop_2": "../logs/2024-09-30/1/", 
    }
    n_fts = 0
    resize = 0
    remove_noisy = False

    # k-fold
    k = 4
    folds_file = "../input/train_folded_v1.csv"  # f"../input/folds_{k}.csv"
    selected_folds = [0, 1, 2, 3]

    # Model
    name = "simple"
    dense_dim = 4096
    layer_dim = 0
    ft = 6 * ("crop" in exp_folders) + 3 * ("dh" in exp_folders) + 3 * ("ch" in exp_folders)
    ft_dim = [
        ft + 3 * len([k for k in exp_folders if "scs" in k]),
        ft + 3 * len([k for k in exp_folders if "nfn" in k]),
        ft + 3 * len([k for k in exp_folders if "ss" in k]),
    ]  # scs, nfn, ss

    p = 0.
    num_classes = len(CLASSES) * 3
    num_classes_aux = 0

    # Training    
    loss_config = {
        "name": "study",
        "weighted": True,
        "use_any": True,
        "smoothing": 0,
        "activation": "study",
        "aux_loss_weight": 0,
        "name_aux": "",
        "smoothing_aux": 0,
        "activation_aux": "",
    }

    data_config = {
        "batch_size": 128,
        "val_bs": 512,
        "mix": "mixup",
        "mix_proba": 0.,
        "sched": False,
        "mix_alpha": 4.,
        "additive_mix": False,
        "num_classes": num_classes,
        "num_classes_aux": num_classes_aux,
        "num_workers": 8,
    }

    optimizer_config = {
        "name": "AdamW",
        "lr": 7e-5,  # 5e-5
        "warmup_prop": 0.,
        "betas": (0.9, 0.999),
        "max_grad_norm": 1.,
        "weight_decay": 1,
    }

    epochs = 15

    use_fp16 = True
    verbose = 1
    verbose_eval = 20

    fullfit = False
    n_fullfit = 1

    local_rank = 0
    distributed = False
    world_size = 1

In [37]:
DEBUG = False
log_folder = None

In [38]:
df = prepare_data_lvl2()

if "fold" not in df.columns:
    folds = pd.read_csv(Config.folds_file)
    df = df.merge(folds, how="left")

# df = df[~df['study_id'].isin([1215498865, 1647904243, 2570933394, 2761048584, 3284652867, 3941522676])].reset_index(drop=True)
# df['any_severe'] = (df[df.columns[3:8]] == 2).max(1)
# df = pd.concat([df, df[df['any_severe']]], ignore_index=True)

In [39]:
if not DEBUG:
    log_folder = prepare_log_folder(LOG_PATH)
    print(f"Logging results to {log_folder}")
    config_df = save_config(Config, log_folder + "config.json")
    create_logger(directory=log_folder, name="logs.txt")

    Config.fullfit = True

preds = k_fold(Config, df, log_folder=log_folder, run=None)

Logging results to ../logs/2024-10-04/43/

-------------   Fold 1 / 4  -------------

    -> 1481 training studies
    -> 494 validation studies
    -> 221193 trainable parameters

Epoch 02/15 (step 0020) 	lr=6.2e-05 	 t=1s 	 loss=0.521    scs_loss=0.261    nfn_loss=0.468    ss_loss=0.575    any_loss=0.241	 val_loss=0.386
Epoch 04/15 (step 0040) 	lr=5.3e-05 	 t=1s 	 loss=0.391    scs_loss=0.237    nfn_loss=0.465    ss_loss=0.544    any_loss=0.239	 val_loss=0.371
Epoch 06/15 (step 0060) 	lr=4.5e-05 	 t=1s 	 loss=0.387    scs_loss=0.235    nfn_loss=0.465    ss_loss=0.542    any_loss=0.236	 val_loss=0.370
Epoch 08/15 (step 0080) 	lr=3.6e-05 	 t=1s 	 loss=0.383    scs_loss=0.235    nfn_loss=0.466    ss_loss=0.542    any_loss=0.235	 val_loss=0.369
Epoch 09/15 (step 0100) 	lr=2.8e-05 	 t=1s 	 loss=0.381    scs_loss=0.236    nfn_loss=0.465    ss_loss=0.543    any_loss=0.236	 val_loss=0.370
Epoch 11/15 (step 0120) 	lr=2.0e-05 	 t=1s 	 loss=0.380    scs_loss=0.236    nfn_loss=0.465    ss_loss=0

### Eval

In [25]:
avg_loss, losses = rsna_loss(df[Config.targets].values, preds, verbose=1)

for k, v in losses.items():
    print(f"- {k}_loss\t: {v:.3f}")

print(f'\n -> CV Score : {avg_loss :.4f}')

tgt any mean 0.185
pred any mean 0.219

- scs_loss	: 0.268
- nfn_loss	: 0.486
- ss_loss	: 0.562
- any_loss	: 0.261

 -> CV Score : 0.3943


In [29]:
df.head()

,study_id,series_id,series_description,spinal_canal_stenosis_l1_l2,spinal_canal_stenosis_l2_l3,spinal_canal_stenosis_l3_l4,spinal_canal_stenosis_l4_l5,spinal_canal_stenosis_l5_s1,left_neural_foraminal_narrowing_l1_l2,left_neural_foraminal_narrowing_l2_l3,...,left_subarticular_stenosis_l2_l3,left_subarticular_stenosis_l3_l4,left_subarticular_stenosis_l4_l5,left_subarticular_stenosis_l5_s1,right_subarticular_stenosis_l1_l2,right_subarticular_stenosis_l2_l3,right_subarticular_stenosis_l3_l4,right_subarticular_stenosis_l4_l5,right_subarticular_stenosis_l5_s1,fold
0,4003253,"[702807833, 1054713880, 2448190387]","[Sagittal T2/STIR, Sagittal T1, Axial T2]",0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
1,4646740,"[3201256954, 3486248476, 3666319702]","[Axial T2, Sagittal T1, Sagittal T2/STIR]",0,0,1,2,0,0,0,...,0,0,2,0,0,1,1,1,0,2
2,7143189,"[132939515, 1951927562, 3219733239]","[Sagittal T2/STIR, Axial T2, Sagittal T1]",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,8785691,"[481125819, 1570286759, 2406919186]","[Sagittal T2/STIR, Sagittal T1, Axial T2]",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,10728036,"[142859125, 2073726394, 2399638375, 3491739931]","[Axial T2, Axial T2, Sagittal T1, Sagittal T2/...",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,3


In [ ]:
# for i in range(5):
#     plt.figure(figsize=(14, 5))
#     for tgt in [1, 2]:
#         plt.subplot(1, 2, tgt)
#         sns.histplot(np.round(preds[:, i, tgt], 1))
#         plt.title(f'{LEVELS[i]} - {tgt} - {(df[Config.targets].values[:, i] == tgt).mean() :.3f}')
#         plt.xlim(-0.01, 1.01)
#     plt.show()

In [ ]:
# df['any_severe'] = (df[df.columns[3:8]] == 2).max(1)
# ids = df[df['any_severe']].index.values

In [ ]:
# avg_loss, losses = rsna_loss(
#     np.concatenate([df[Config.targets].values, df[Config.targets].values[ids]]),
#     np.concatenate([preds, preds[ids]]),
# )

# for k, v in losses.items():
#     print(f"- {k}_loss\t: {v:.3f}")

# print(f'\n -> CV Score : {avg_loss :.4f}')

Done ! 